In [341]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

## Connecting to existing database and table within the database

In [6]:
client = MongoClient()

In [7]:

db = client.images2
collection= db.image_data

In [8]:
print db
print collection

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'images2')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'images2'), u'image_data')


## Test connection

In [9]:
#Testing retreving data from Mongodb. Imp for when an img-no doesnt exist
cur = collection.find({'img_no':29},{'_id':0,'img_no':1,'prod_info_table':1,'prod_desc':1,"title":1})
cur.rewind()

try:
    print (cur)[0]['title']
except:
    print "Not readable error"
    

Invicta Men's 8932 Pro Diver Collection Silver-Tone Watch


In [10]:
cur[0]['prod_desc']

u'Big, bold and masculine, the Invicta Quartz Pro Diver 8932 is an affordable luxury watch with classically styled features. This precisely constructed quartz timepiece for men features a solid stainless steel case that\'s 37mm wide, slightly smaller than its 8926 cousin. It has a black face with Tritnite luminous hands and markers and a unidirectional black bezel with white numerals and markings. Other features include anti-reflective mineral crystal, date display at 3 o\'clock, and water resistance to 200 meters. The highly polished stainless steel band has a diver buckle with safety clasp.Pro Diver Collection Plunge into any horizon using the steadfast guidance of the Invicta Pro Diver. Stylishly classic, internal workings are forged with variations of either Swiss chronograph or 21-jewel automatic movements and willingly navigate in depths up to 300 meters. Built with confident prowess, the fortitude with which these timepieces function makes the Pro Diver the quintessential in per

## Getting the information from MongoDB

In [12]:
#total number of docs in collections
total_docs = collection.count()

In [13]:
#getting the data(img_no and prod_info_table) out of Mongodb
#5940 is the max img_no known through looking at data
prod_desc_table = []

for i in xrange(5940):
    cursor = collection.find({'img_no':i},{'_id':0,'img_no':1,'prod_info_table':1,'prod_desc':1,'title':1})
    #print cursor[0]
    cursor.rewind()
    try: 
        #We do cursor of 0 because cursor only has 1 result a dictionary. 
        #We are adding the img_no as another feature in the prod_info_table dictionary
        #This is to keep track of the image number for each row . Will be used in retirving the results when we know th
        #nearest neigbours for the model. We also add product description and title for future TFIDF
        #print cursor[0]    
        prod_dict = cursor[0]["prod_info_table"]
        
        #Insert the three in the exsiting dictionary to get one comprehensive dict for dataframe
        prod_dict['Img_no'] = cursor[0]["img_no"]
        prod_dict['Prod_description'] = cursor[0]["prod_desc"]
        prod_dict['Prod_title'] = cursor[0]["title"]
        prod_desc_table.append(prod_dict)
    
        
    except:
        pass # Some number might not exist in between due to broken links while scraping. 
        #This will ensure we move to the next image number without raising an error in that case

In [15]:
len(prod_desc_table)

4957

##  Pandas dataframe and EDA

### Loading dictionary to pandas dataframe

In [206]:
df = pd.DataFrame(prod_desc_table)

In [207]:
len(df.columns)

29

In [208]:
df.columns

Index([                       u'Band Color',
                           u'Band Material',
                             u'Band length',
                              u'Band width',
                     u'Battery description',
                          u'Bezel material',
       u'Brand, Seller, or Collection Name',
                          u'Case Thickness',
                           u'Case diameter',
                           u'Case material',
                                   u'Clasp',
                              u'Dial color',
                            u'Display Type',
                                  u'Height',
                                  u'Img_no',
                             u'Item Length',
                              u'Item Shape',
                             u'Item weight',
                                  u'Length',
                                u'Material',
                             u'Metal stamp',
                              u'Model Year',
          

In [102]:
df.iloc[:,:10].head()

,Band Color,Band Material,Band length,Band width,Battery description,Bezel material,"Brand, Seller, or Collection Name",Case Thickness,Case diameter,Case material
0,Black,Resin,Men's Standard,22 millimeters,NaN,Resin,Casio,12 millimeters,43 millimeters,Resin
1,Black,Calfskin,Mens Standard,18 millimeters,NaN,Brass,Timex,8 millimeters,35 millimeters,Brass
2,Black,Resin,mens,18 millimeters,NaN,Plastic,Casio,8 millimeters,35 millimeters,Plastic
3,Black,Resin,mens,20 millimeters,NaN,Resin,Casio,7.8 millimeters,35 millimeters,Resin
4,Biege,Nylon,Mens-Standard,20 millimeters,NaN,Stainless Steel,Casio,11 millimeters,41 millimeters,Resin


In [209]:
df.iloc[:,10:20].head()

,Clasp,Dial color,Display Type,Height,Img_no,Item Length,Item Shape,Item weight,Length,Material
0,Buckle,Black,Analog,NaN,0,NaN,Round,2.08 Ounces,NaN,NaN
1,Buckle,White,Analog,NaN,1,NaN,Round,1.44 Ounces,NaN,NaN
2,Buckle,White,Analog,NaN,2,NaN,Round,18.14 Grams,NaN,NaN
3,Buckle,White,Analog,NaN,3,NaN,Round,NaN,NaN,NaN
4,Buckle,Beige,Analog,NaN,4,NaN,Round,16 Ounces,NaN,NaN


In [210]:
df.iloc[:,20:].head()

,Metal stamp,Model Year,Model number,Part Number,Prod_description,Prod_title,Special features,Water pressure resistance,Water resistant depth
0,NaN,2014,MRW200H-1BV,MRW200H-1BV,This watch from Casio is water resistant up to...,Casio Men's MRW200H-1BCT,Water Resistant,NaN,330 Feet
1,NaN,2009,T2H281,T2H2819J,Not available,Timex Men's Easy Reader Watch,"measures-seconds, Light",NaN,99 Feet
2,NaN,2014,MQ24-7B2,MQ24-7B2,Splash Resistant / Round White Face / Black Re...,Casio Men's MQ24-7B2 Analog Watch with Black R...,Water Resistant,NaN,NaN
3,NaN,NaN,MQ24-7B,MQ24-7B,The basic black-and-white design of the Casio ...,Casio Unisex MQ24-7B Analog Black Resin Watch,Water Resistant,NaN,50
4,NaN,2014,FT-500WC-5BVCF,FT-500WC-5BVCF,This watch from the Forester collection featur...,Casio Men's FT500WVB-5BV,Luminous,NaN,330 Feet


### Cleaning data

In [211]:
df.isnull().sum(axis =0)

Band Color                             83
Band Material                          33
Band length                           197
Band width                             72
Battery description                  4956
Bezel material                        331
Brand, Seller, or Collection Name       0
Case Thickness                        105
Case diameter                          29
Case material                         119
Clasp                                1768
Dial color                             21
Display Type                           72
Height                               4956
Img_no                                  0
Item Length                          4955
Item Shape                             73
Item weight                          2617
Length                               4956
Material                             4955
Metal stamp                          4460
Model Year                           1780
Model number                           34
Part Number                       

In [212]:
# Anything with more than 50% null values is dropped immediately 
df.drop(['Battery description','Water pressure resistance','Material','Height','Item Length','Metal stamp','Length','Item weight'],axis =1,inplace = True)

In [213]:
df.shape

(4957, 21)

In [214]:
df.head(2)

,Band Color,Band Material,Band length,Band width,Bezel material,"Brand, Seller, or Collection Name",Case Thickness,Case diameter,Case material,Clasp,...,Display Type,Img_no,Item Shape,Model Year,Model number,Part Number,Prod_description,Prod_title,Special features,Water resistant depth
0,Black,Resin,Men's Standard,22 millimeters,Resin,Casio,12 millimeters,43 millimeters,Resin,Buckle,...,Analog,0,Round,2014,MRW200H-1BV,MRW200H-1BV,This watch from Casio is water resistant up to...,Casio Men's MRW200H-1BCT,Water Resistant,330 Feet
1,Black,Calfskin,Mens Standard,18 millimeters,Brass,Timex,8 millimeters,35 millimeters,Brass,Buckle,...,Analog,1,Round,2009,T2H281,T2H2819J,Not available,Timex Men's Easy Reader Watch,"measures-seconds, Light",99 Feet


In [215]:
#Remove Model number and part number beacuase they arent giving us useful info(plus already in title)
df.drop(['Model number','Part Number'],axis =1,inplace = True)

In [216]:
print df.columns
df.shape

Index([                       u'Band Color',
                           u'Band Material',
                             u'Band length',
                              u'Band width',
                          u'Bezel material',
       u'Brand, Seller, or Collection Name',
                          u'Case Thickness',
                           u'Case diameter',
                           u'Case material',
                                   u'Clasp',
                              u'Dial color',
                            u'Display Type',
                                  u'Img_no',
                              u'Item Shape',
                              u'Model Year',
                        u'Prod_description',
                              u'Prod_title',
                        u'Special features',
                   u'Water resistant depth'],
      dtype='object')


(4957, 19)

We still have plenty of null values but we'll deal with them with them as we go along.
Since we apply to tfidf to categorical part we'll start by dealing with numerical data seperately. We will later combine the data together to form a feature space   


In [231]:
dfnum = df[['Band width','Water resistant depth','Case diameter','Case Thickness','Img_no']]
dfcat = df.drop(['Band width','Water resistant depth','Case diameter','Case Thickness'],axis=1)

### Converting all the numerical features into numbers




In [261]:
#Ensuring that all units are in the same case(lower).Others are already lowercase

dfnum['Water resistant depth'] = map(lambda x: x.lower(),dfnum['Water resistant depth'])


dfnum.head()

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Band width,Water resistant depth,Case diameter,Case Thickness,Img_no
0,22 millimeters,330 feet,43 millimeters,12 millimeters,0
1,18 millimeters,99 feet,35 millimeters,8 millimeters,1
2,18 millimeters,0 feet,35 millimeters,8 millimeters,2
3,20 millimeters,50,35 millimeters,7.8 millimeters,3
4,20 millimeters,330 feet,41 millimeters,11 millimeters,4


In [232]:
#There are 5 columns with numbers. Filling na with (0 mm, ounces,feet )Looking for the different numerical units in each of them
def fill_nulls(col_name,units):
    '''
    Input: Column name and units to be added
    Output: None
    '''
    print "No of nulls before filling for %s are"%col_name, sum(dfnum[col_name].isnull())
    dfnum[col_name].fillna( '0 %s'%units , inplace = True)
    print "No of nulls now",sum(dfnum[col_name].isnull())
    

In [262]:
dfnum.head()

,Band width,Water resistant depth,Case diameter,Case Thickness,Img_no
0,22 millimeters,330 feet,43 millimeters,12 millimeters,0
1,18 millimeters,99 feet,35 millimeters,8 millimeters,1
2,18 millimeters,0 feet,35 millimeters,8 millimeters,2
3,20 millimeters,50,35 millimeters,7.8 millimeters,3
4,20 millimeters,330 feet,41 millimeters,11 millimeters,4


In [236]:
fill_nulls("Band width", 'millimeters')
fill_nulls("Case Thickness", 'millimeters')
fill_nulls("Case diameter", 'millimeters')

No of nulls before filling for Band width are 0
No of nulls now 0
No of nulls before filling for Case Thickness are 0
No of nulls now 0
No of nulls before filling for Case diameter are 0
No of nulls now 0


In [263]:
fill_nulls('Water resistant depth','feet')


No of nulls before filling for Water resistant depth are 0
No of nulls now 0


In [264]:
dfnum.head(1)

,Band width,Water resistant depth,Case diameter,Case Thickness,Img_no
0,22 millimeters,330 feet,43 millimeters,12 millimeters,0


In [293]:
# Function to get unique units per column to study them
'''
Input: Column Name
Output: Returns unique values of units used

'''


def get_unique_num_units(col_name):
    units = []
    for x in dfnum[col_name]:
        try :
            units.append( str(x).split()[1])
        except:
            pass
    u =set(units)
    return u

In [294]:
get_unique_num_units("Band width")

set()

#### Getting the unit conversions in order

In [295]:
from pint import UnitRegistry

In [296]:
ureg = UnitRegistry()

In [297]:
def unit_conversion_pinthelper(ureg,elem, units_to):
    '''
    Takes a single entry as input and converts it to desired units. If the element doesnt have units,
    except adds the desired units to it.
    
    Input: UnitRegistery object ,column name, units to convert to(an assumption made viewing the data)
    
    Output: A numpy array of data type quantity that can replace the entire 
    
    '''
    try: 
        elem = ureg.Quantity(elem).to(units_to)
    except:
        elem = ureg.Quantity(elem, units_to)
    return  elem

def unit_conversion(colname,desired_units):
    dfnum[colname] = map(lambda x:float(unit_conversions(ureg,x,desired_units).magnitude),dfnum[colname])
    

example of conversion
df["Band width"] = map(lambda x:float(unit_conversion_pinthelper(ureg,x,"millimeter").magnitude),df["Band width"])

In [298]:
unit_conversion("Band width","millimeter")
unit_conversion("Case diameter","millimeter")   
unit_conversion("Case Thickness","millimeter")


/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [299]:
unit_conversion('Water resistant depth',"millimeter")

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [300]:
dfnum.head()

,Band width,Water resistant depth,Case diameter,Case Thickness,Img_no
0,22,100584.0,43,12.0,0
1,18,30175.2,35,8.0,1
2,18,0.0,35,8.0,2
3,20,50.0,35,7.8,3
4,20,100584.0,41,11.0,4


With this our numerical varibales are ready. Now for categorical variables

## Dealing with categorical variables

In [383]:
dfcat.dtypes

Band Color                           object
Band Material                        object
Band length                          object
Bezel material                       object
Brand, Seller, or Collection Name    object
Case material                        object
Clasp                                object
Dial color                           object
Display Type                         object
Img_no                                int64
Item Shape                           object
Model Year                           object
Prod_description                     object
Prod_title                           object
Special features                     object
Final_String                         object
dtype: object

In [394]:
columns_df = dfcat.drop(['Img_no',"Prod_description","Final_String"],axis=1).columns #Product description column needs encoding beacuse its 
#it has non ascii characters that cant be converted to string by str

In [404]:
#Converting all columns into string datatype
for col in columns_df:
    dfcat[col] = map(lambda x: str(x).lower(),dfcat[col])
    print "%s done"%col
# Now we move on to product description tab

Band Color done
Band Material done
Band length done
Bezel material done
Brand, Seller, or Collection Name done
Case material done
Clasp done
Dial color done
Display Type done
Item Shape done
Model Year done
Prod_title done
Special features done


In [388]:
dfcat.dtypes

Band Color                           object
Band Material                        object
Band length                          object
Bezel material                       object
Brand, Seller, or Collection Name    object
Case material                        object
Clasp                                object
Dial color                           object
Display Type                         object
Img_no                                int64
Item Shape                           object
Model Year                           object
Prod_description                     object
Prod_title                           object
Special features                     object
Final_String                         object
dtype: object

### Getting data ready for TFIDF

In [319]:
#Getting all columns except image number which is a numerical column which should serve as our index
columns_df = dfcat.drop(['Img_no'],axis=1).columns

In [328]:
#Adding up all the strings
dfcat["Final_String"] = ""
for col in columns_df:
    dfcat["Final_String"] += " "
    dfcat["Final_String"] += dfcat[col] 

In [337]:
print dfcat['Final_String'][8], ">>>>",dfcat["Prod_description"][8]
    

 Brown Canvas mens Stainless steel Casio Stainless steel Hook-and-loop Beige Analog and digital Round 2014 Watch features an analog/digital combination display, 30-record telememo, world time (30 cities), LED light, 1/100 second stopwatch, countdown timer, three daily alarms, hourly time signal, auto calendar, 12- and 24- hour formats, 10-year battery life and water resistance to 50 meters. Casio Men's AW80V-5BV alarm-feature, Water Resistant, World time >>>> Watch features an analog/digital combination display, 30-record telememo, world time (30 cities), LED light, 1/100 second stopwatch, countdown timer, three daily alarms, hourly time signal, auto calendar, 12- and 24- hour formats, 10-year battery life and water resistance to 50 meters.


In [338]:
#Seems like its working. Now creating our dataframe for tfidf
X = dfcat[["Img_no","Final_String"]]

In [364]:
X[[]]

(4957, 2)

In [380]:
# Lowercasing the string
data = (X['Final_String'].reshape(-1,1)).shape

In [381]:


vect = TfidfVectorizer(stop_words = "english")
tfidf = vect.fit_transform(data)



AttributeError: 'int' object has no attribute 'lower'

In [375]:
tfidf.todense().shape

(2, 2)

In [370]:
X[['Final_String']]

,Final_String
0,Black Resin Men's Standard Resin Casio Resin ...
1,Black Calfskin Mens Standard Brass Timex Bras...
2,Black Resin mens Plastic Casio Plastic Buckle...
3,Black Resin mens Resin Casio Resin Buckle Whi...
4,Biege Nylon Mens-Standard Stainless Steel Cas...
5,Various Resin mens Resin Casio Resin Buckle D...
6,Silver Stainless steel mens Stainless steel C...
7,Various Resin Mens Resin Casio Resin Buckle B...
8,Brown Canvas mens Stainless steel Casio Stain...
9,Silver Stainless steel mens Brass Timex Brass...


In [170]:
len_max_ind = 0 # index of element with Max Length of table
len_max = 0 # max length
ind = [] # All the indexes with max length to compare if table has same c
# Getting the table with max components to be used to derive the table contents. The one which has the max

for i,result in enumerate(results):
    if len(result[1]) > len_max :
        len_max = len(result[1])
        len_max_index = i
        
    elif  (len(result[1]) == len_max):
        ind.append((i,len_max))
        
    else:
        pass

In [178]:
print len_max_index,len_max
print ind

1213 25
[(1, 23), (3, 23), (5, 23), (14, 24), (31, 24), (43, 24), (44, 24), (63, 24), (68, 24), (69, 24), (81, 24), (85, 24), (89, 24), (94, 24), (108, 24), (115, 24), (119, 24), (122, 24), (152, 24), (168, 24), (179, 24), (193, 24), (199, 24), (254, 24), (270, 24), (272, 24), (284, 24), (302, 24), (314, 24), (343, 24), (360, 24), (401, 24), (408, 24), (412, 24), (420, 24), (428, 24), (435, 24), (439, 24), (457, 24), (485, 24), (503, 24), (510, 24), (546, 24), (550, 24), (574, 24), (619, 24), (621, 24), (635, 24), (658, 24), (661, 24), (695, 24), (699, 24), (727, 24), (737, 24), (743, 24), (744, 24), (748, 24), (828, 24), (834, 24), (845, 24), (862, 24), (863, 24), (872, 24), (874, 24), (905, 24), (906, 24), (911, 24), (917, 24), (959, 24), (963, 24), (973, 24), (974, 24), (1036, 24), (1071, 24), (1078, 24), (1087, 24), (1088, 24), (1093, 24), (1127, 24), (1134, 24), (1137, 24), (1142, 24), (1156, 24), (1157, 24), (1158, 24), (1159, 24), (1181, 24), (1206, 24), (1212, 24)]


We can see above that  index 1213 has the largest number of table contents we'll focus on that to get the 
table contents

In [224]:
#Extracting the Column names
prod_table = (results[1213][1])
products = []
column_names = []
for i in prod_table:
    column_names.append(str(i.rsplit(', u')[0]))
    products.append(i.rsplit(', u')[1])
column_names

["(u'Brand, Seller, or Collection Name'",
 "(u'Model number'",
 "(u'Part Number'",
 "(u'Model Year'",
 "(u'Item Shape'",
 "(u'Dial window material type\\ufeff'",
 "(u'Display Type'",
 "(u'Clasp'",
 "(u'Metal stamp'",
 "(u'Case material'",
 "(u'Case diameter'",
 "(u'Case Thickness'",
 "(u'Band Material'",
 "(u'Band length'",
 "(u'Band width'",
 "(u'Band Color'",
 "(u'Dial color'",
 "(u'Bezel material'",
 "(u'Bezel function\\ufeff'",
 "(u'Calendar\\ufeff'",
 "(u'Special features'",
 "(u'Item weight'",
 "(u'Movement\\ufeff'",
 "(u'Water pressure resistance'",
 "(u'Water resistant depth'"]

column_names

In [221]:
columns = []
for col in column_names:
    columns.append(((col.replace("(","")).replace("u'","")).replace("'",""))

In [222]:
columns

['Brand, Seller, or Collection Name',
 'Model number',
 'Part Number',
 'Model Year',
 'Item Shape',
 'Dial window material type\\ufeff',
 'Display Type',
 'Clasp',
 'Metal stamp',
 'Case material',
 'Case diameter',
 'Case Thickness',
 'Band Material',
 'Band length',
 'Band width',
 'Band Color',
 'Dial color',
 'Bezel material',
 'Bezel function\\ufeff',
 'Calendar\\ufeff',
 'Special features',
 'Item weight',
 'Movement\\ufeff',
 'Water pressure resistance',
 'Water resistant depth']

In [225]:
products

[u"'Citizen')",
 u"'AT4004-52E')",
 u"'AT4004-52E')",
 u"'2011')",
 u"'Round')",
 u"'Anti reflective sapphire')",
 u"'Analog')",
 u"'Fold-Over Clasp with Hidden Double Push-Button\\ufeff')",
 u"'None')",
 u"'Stainless steel')",
 u"'42 millimeters')",
 u"'13 millimeters')",
 u"'Two-tone stainless steel')",
 u'"Men\'s Standard")',
 u"'24 millimeters')",
 u"'Silver')",
 u"'Black')",
 u"'Stainless steel')",
 u"'Stationary')",
 u"'Perpetual calendar')",
 u"'Radio controlled, Chronograph, Luminous, Stop watch, Tachometer')",
 u"'6.24 Ounces')",
 u"'Japanese quartz')",
 u"'20 bar')",
 u"'660 Feet')"]

In [315]:
pd.DataFrame(results[1][1])

ValueError: If using all scalar values, you must pass an index

In [231]:
import json
for i in  prod_table:
    json.loads(i)

ValueError: No JSON object could be decoded

In [247]:
db2 = client.images2
col2 = db2.image_data



In [248]:
x = col2.find_one()['prod_info_table']

In [249]:
for i in x:
    print i.keys()
    

[u'Brand, Seller, or Collection Name']
[u'Model number']
[u'Part Number']
[u'Model Year']
[u'Item Shape']
[u'Display Type']
[u'Clasp']
[u'Case material']
[u'Case diameter']
[u'Case Thickness']
[u'Band Material']
[u'Band length']
[u'Band width']
[u'Band Color']
[u'Dial color']
[u'Bezel material']
[u'Special features']
[u'Item weight']
[u'Water resistant depth']
